In [25]:
# import libraries
import boto3
import pandas as pd
from pymongo import MongoClient
import csv

In [26]:
# Assign the s3 bucket to a variable
bucket_name = 'data-eng-204-final-project'
folder_path = 'Academy/'

def get_s3_objects(bucket_name, folder_path):
    """
    Returns a Pandas DataFrame by reading CSV files from a specified S3 bucket and prefix.

    Args:
        bucket_name (str): The name of the S3 bucket.
        folder_path (str): The prefix of the S3 objects to read.

    Returns:
        pandas.DataFrame: A merged Pandas DataFrame containing the contents of all the CSV files in the specified S3 bucket and prefix.
    """
    # Create an S3 client object
    s3_client = boto3.client('s3')

    # List all objects in the specified S3 bucket and prefix
    objects = s3_client.list_objects(Bucket=bucket_name, Prefix=folder_path)

    # Extract the keys of all CSV objects in the S3 bucket and prefix
    keys = [obj['Key'] for obj in objects['Contents'] if obj['Key']]

    # Create a list to store Pandas DataFrames for each CSV object
    dfs = []

    # Iterate over each CSV object, read it into a Pandas DataFrame, and append it to the list of DataFrames
    for key in keys:
        # Get the CSV object from S3
        obj = s3_client.get_object(Bucket=bucket_name, Key=key)

        # Read the CSV object into a Pandas DataFrame
        df = pd.read_csv(obj['Body'], delimiter=',')

        # Add a column to the DataFrame with the name of the file
        df['filename'] = key.split('/')[-1]

        # Append the DataFrame to the list of DataFrames
        dfs.append(df)

    # Concatenate all the DataFrames into a single DataFrame
    final_df = pd.concat(dfs, ignore_index=True)

    # Return the merged DataFrame
    return final_df

raw_df = get_s3_objects(bucket_name,folder_path)
raw_df

,name,trainer,Analytic_W1,Independent_W1,Determined_W1,Professional_W1,Studious_W1,Imaginative_W1,Analytic_W2,Independent_W2,...,Determined_W9,Professional_W9,Studious_W9,Imaginative_W9,Analytic_W10,Independent_W10,Determined_W10,Professional_W10,Studious_W10,Imaginative_W10
0,Quintus Penella,Gregor Gomez,1,2,2,1,2,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Simon Murrey,Gregor Gomez,6,1,1,2,4,2,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gustaf Lude,Gregor Gomez,6,4,1,1,2,3,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Yolanda Fosse,Gregor Gomez,2,1,2,3,3,3,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lynnett Swin,Gregor Gomez,2,2,4,5,1,2,3.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Moritz Mosedall,Mohammad Velazquez,1,1,5,1,2,6,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,Chaim Inseal,Mohammad Velazquez,1,3,3,4,1,2,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
394,Gertruda Syddie,Mohammad Velazquez,3,1,2,8,1,4,2.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,Thom Derwin,Mohammad Velazquez,3,7,3,3,3,1,2.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
def get_course_dataframe(bucket_name, course_name):
    """
    Returns a Pandas DataFrame containing data from all CSV files corresponding to a specified course in an S3 bucket.

    Args:
        bucket_name (str): The name of the S3 bucket.
        course_name (str): The name of the course.

    Returns:
        pandas.DataFrame: A merged Pandas DataFrame containing the contents of all the CSV files corresponding to the specified course in the specified S3 bucket.
    """
    # Define the prefix of the S3 objects for the specified course
    folder_path = f'Academy/{course_name}'

    # Get a Pandas DataFrame by reading CSV files from the specified S3 bucket and prefix
    df = get_s3_objects(bucket_name, folder_path)

    # Extract course and started_date information from the filename and add them as separate columns
    df['course'] = df['filename'].str.extract(r'^(.*?)_')
    df['start_date'] = df['filename'].str.extract(r'_(\d{4}-\d{2}-\d{2})\.csv$')

    # Reorder the columns of the DataFrame for readability and consistency
    cols = df.columns.tolist()
    cols.remove('trainer')
    cols.insert(1, 'trainer')
    cols.remove('course')
    cols.insert(2, 'course')
    cols.remove('start_date')
    cols.insert(3, 'start_date')
    df = df.reindex(columns=cols)

    # Return the updated DataFrame
    return df

df = get_course_dataframe('data-eng-204-final-project', 'Data')
df.head()

,name,trainer,course,start_date,Analytic_W1,Independent_W1,Determined_W1,Professional_W1,Studious_W1,Imaginative_W1,...,Determined_W9,Professional_W9,Studious_W9,Imaginative_W9,Analytic_W10,Independent_W10,Determined_W10,Professional_W10,Studious_W10,Imaginative_W10
0,Pyotr De Zuani,Trixie Orange,Data,2019-02-18,1,3,4,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Vince Scott,Trixie Orange,Data,2019-02-18,1,1,4,3,3,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Kellie Althorp,Trixie Orange,Data,2019-02-18,3,4,1,1,5,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aida Bothams,Trixie Orange,Data,2019-02-18,4,2,2,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nettie Civitillo,Trixie Orange,Data,2019-02-18,5,4,3,1,1,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
def get_all_courses_dataframe(bucket_name):
    """
    Returns a merged Pandas DataFrame containing data from all CSV files corresponding to all courses in an S3 bucket.

    Args:
        bucket_name (str): The name of the S3 bucket.

    Returns:
        pandas.DataFrame: A merged Pandas DataFrame containing the contents of all the CSV files corresponding to all the courses in the specified S3 bucket.
    """
    # Get a Pandas DataFrame for each course and concatenate them into a single DataFrame
    df_business = get_course_dataframe(bucket_name, 'Business')
    df_data = get_course_dataframe(bucket_name, 'Data')
    df_eng = get_course_dataframe(bucket_name, 'Engineering')
    df_all_courses = pd.concat([df_business, df_data, df_eng], ignore_index=True)

    # Return the merged DataFrame
    return df_all_courses

all_courses = get_all_courses_dataframe('data-eng-204-final-project')
all_courses

,name,trainer,course,start_date,Analytic_W1,Independent_W1,Determined_W1,Professional_W1,Studious_W1,Imaginative_W1,...,Determined_W9,Professional_W9,Studious_W9,Imaginative_W9,Analytic_W10,Independent_W10,Determined_W10,Professional_W10,Studious_W10,Imaginative_W10
0,Quintus Penella,Gregor Gomez,Business,2019-02-11,1,2,2,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Simon Murrey,Gregor Gomez,Business,2019-02-11,6,1,1,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gustaf Lude,Gregor Gomez,Business,2019-02-11,6,4,1,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Yolanda Fosse,Gregor Gomez,Business,2019-02-11,2,1,2,3,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lynnett Swin,Gregor Gomez,Business,2019-02-11,2,2,4,5,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,Moritz Mosedall,Mohammad Velazquez,Engineering,2019-12-30,1,1,5,1,2,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,Chaim Inseal,Mohammad Velazquez,Engineering,2019-12-30,1,3,3,4,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
394,Gertruda Syddie,Mohammad Velazquez,Engineering,2019-12-30,3,1,2,8,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
395,Thom Derwin,Mohammad Velazquez,Engineering,2019-12-30,3,7,3,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
def normalise_scores_df(all_courses):
    """
    Normalizes the scores in the input DataFrame by melting the DataFrame and separating out the 'trait' and 'week' columns.

    Args:
        all_courses (pandas.DataFrame): The input DataFrame containing course scores.

    Returns:
        pandas.DataFrame: The normalized DataFrame containing scores for each trait in each week for each student in each course.
    """
    # Define the weeks and traits to be used in the DataFrame
    weeks = [f'W{i}' for i in range(1, 11)]
    traits = ['Analytic', 'Independent', 'Determined', 'Professional', 'Studious', 'Imaginative']
    
    # Define the id and value columns to be used in the melted DataFrame
    id_vars = ['name', 'trainer', 'course', 'start_date']
    value_vars = [f'{trait}_{week}' for trait in traits for week in weeks]
    
    # Melt the input DataFrame and separate out the 'trait' and 'week' columns
    scores_df = pd.melt(all_courses, id_vars=id_vars, value_vars=value_vars, var_name='trait_week', value_name='score')
    scores_df[['trait', 'week']] = scores_df['trait_week'].str.split('_W', expand=True)
    scores_df.drop('trait_week', axis=1, inplace=True)
    
    # Return the normalised DataFrame
    return scores_df

normalised_df = normalise_scores_df(all_courses)
normalised_df

,name,trainer,course,start_date,score,trait,week
0,Quintus Penella,Gregor Gomez,Business,2019-02-11,1.0,Analytic,1
1,Simon Murrey,Gregor Gomez,Business,2019-02-11,6.0,Analytic,1
2,Gustaf Lude,Gregor Gomez,Business,2019-02-11,6.0,Analytic,1
3,Yolanda Fosse,Gregor Gomez,Business,2019-02-11,2.0,Analytic,1
4,Lynnett Swin,Gregor Gomez,Business,2019-02-11,2.0,Analytic,1
...,...,...,...,...,...,...,...
23815,Moritz Mosedall,Mohammad Velazquez,Engineering,2019-12-30,NaN,Imaginative,10
23816,Chaim Inseal,Mohammad Velazquez,Engineering,2019-12-30,NaN,Imaginative,10
23817,Gertruda Syddie,Mohammad Velazquez,Engineering,2019-12-30,NaN,Imaginative,10
23818,Thom Derwin,Mohammad Velazquez,Engineering,2019-12-30,NaN,Imaginative,10


In [30]:
# Course enrollment date
normalised_df['start_date'].unique()

array(['2019-02-11', '2019-03-18', '2019-04-15', '2019-05-20',
       '2019-07-15', '2019-07-29', '2019-08-12', '2019-09-16',
       '2019-10-21', '2019-11-18', '2019-12-30', '2019-02-18',
       '2019-03-04', '2019-04-08', '2019-07-22', '2019-08-05',
       '2019-08-19', '2019-09-23', '2019-10-28', '2019-12-16',
       '2019-04-01', '2019-04-29', '2019-05-27', '2019-11-25'],
      dtype=object)

In [31]:
# Course enrollment counts
normalised_df[['course']].value_counts()

course     
Engineering    8340
Data           8220
Business       7260
dtype: int64

In [32]:
# Mean score values sorted by course
normalised_df.groupby('course')['score'].mean().sort_values(ascending=False)

course
Data           5.342416
Engineering    5.283641
Business       5.170643
Name: score, dtype: float64

In [33]:
# Uniques trainer name
normalised_df['trainer'].unique()

array(['Gregor Gomez', 'Bruce Lugo', 'Neil Mccarthy', 'Rachel Richard',
       'Hamzah Melia', 'Burhan Milner', 'Elly Kelly', 'Ely Kely',
       'Trixie Orange', 'John Sandbox', 'Edward Reinhart', 'Lucy Foster',
       'Gina Cartwright', 'Eshal Brandt', 'Macey Broughton',
       'Igor Coates', 'Mohammad Velazquez', 'Martina Meadows'],
      dtype=object)

In [34]:
# Correct the trainer name anomalies
normalised_df['trainer'] = normalised_df['trainer'].replace('Ely Kely', 'Elly Kelly')
normalised_df['trainer'].unique()

array(['Gregor Gomez', 'Bruce Lugo', 'Neil Mccarthy', 'Rachel Richard',
       'Hamzah Melia', 'Burhan Milner', 'Elly Kelly', 'Trixie Orange',
       'John Sandbox', 'Edward Reinhart', 'Lucy Foster',
       'Gina Cartwright', 'Eshal Brandt', 'Macey Broughton',
       'Igor Coates', 'Mohammad Velazquez', 'Martina Meadows'],
      dtype=object)

In [35]:
# Mean score values by trainer
normalised_df.groupby('trainer').mean().sort_values(by=['score'],ascending=False)

/var/folders/jr/dhzz03zd14b_2cf35k7btydh0000gn/T/ipykernel_9588/3329588578.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normalised_df.groupby('trainer').mean().sort_values(by=['score'],ascending=False)


,score
trainer,
Lucy Foster,5.542977
Igor Coates,5.488462
Eshal Brandt,5.369198
Gina Cartwright,5.307018
Elly Kelly,5.300000
Macey Broughton,5.276803
John Sandbox,5.254438
Bruce Lugo,5.232297
Gregor Gomez,5.211382


In [36]:
# Mean score values by trainee name
normalised_df.groupby('name').mean().sort_values(by=['score'],ascending=False)

/var/folders/jr/dhzz03zd14b_2cf35k7btydh0000gn/T/ipykernel_9588/182064997.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  normalised_df.groupby('name').mean().sort_values(by=['score'],ascending=False)


,score
name,
Violet Luscombe,6.383333
Gabbey Caesman,6.116667
Yalonda Beacom,6.083333
Auberon Werny,6.066667
Reggie Lawlor,6.033333
...,...
Quintus Penella,1.666667
Connor Gegg,1.666667
Teirtza Docharty,1.666667


In [37]:
normalised_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23820 entries, 0 to 23819
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        23820 non-null  object 
 1   trainer     23820 non-null  object 
 2   course      23820 non-null  object 
 3   start_date  23820 non-null  object 
 4   score       19542 non-null  float64
 5   trait       23820 non-null  object 
 6   week        23820 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.3+ MB


In [38]:
normalised_df['start_date'] = pd.to_datetime(normalised_df['start_date'])
normalised_df['week'] = pd.to_numeric(normalised_df['week'])
normalised_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23820 entries, 0 to 23819
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   name        23820 non-null  object        
 1   trainer     23820 non-null  object        
 2   course      23820 non-null  object        
 3   start_date  23820 non-null  datetime64[ns]
 4   score       19542 non-null  float64       
 5   trait       23820 non-null  object        
 6   week        23820 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 1.3+ MB


In [39]:
# Save the dataframe into a csv file
normalised_df.to_csv('academy_clean.csv')

In [40]:
# Connect to MongoDB server
client = MongoClient('mongodb://localhost:27017/')
# Select the database
db = client['Sparta']
# Select the collection
collection = db['Academy']

# Create an empty dictionary to store the scores
scores_dict = {}

filename = 'academy_clean.csv'

with open(filename) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # Skip header row
    header = next(csv_reader)
    # Loop through each row in the csv file
    for row in csv_reader:
        # Extract the fields from the row
        name = row[1]
        trainer = row[2]
        course = row[3]
        start_date = row[4]
        score = row[5]
        trait = row[6]
        week = row[7]
        # If the name is not already in the dictionary, add it
        if name not in scores_dict:
            scores_dict[name] = {'trainer': trainer, 'start_date': start_date, 'course': course, 'week':{}}
        # If the course is not already in the dictionary, add it
        if course not in scores_dict[name]['course']:
            scores_dict[name]['course'].append(course)
        # If the week is not already in the dictionary, add it
        if week not in scores_dict[name]['week']:
            scores_dict[name]['week'][week] = {}
        # Convert the score to float if it exists or None if it's not a float
        try:
            score = float(score)
        except (ValueError, TypeError):
            score = None
        # Add the score for the trait and week
        scores_dict[name]['week'][week][trait] = score

# Convert the dictionary to a list of documents
documents = []
for name, data in scores_dict.items():
    documents.append({'name': name, 'trainer': data['trainer'], 'start_date': data['start_date'], 'course': data['course'], 'week': data['week']})

# Insert the scores into the MongoDB collection
collection.insert_many(documents)